In [1]:
import os
import pyspark
import numpy as np
import pandas as pd
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

In [2]:
from util import get_data

In [3]:
def get_eda(data, feature_list, feature_family=None):
    """
    Perform EDA on the provided dataset base on its family i.e categorical or continuos

    Parameters:
        data: DataFrame (pySpark)
        feature_list (list)
        feature_family: continuous or categorical, default:None

    Returns:
        DataFrame (pySpark)
    """
    if feature_family == "continuos":
        from pyspark.sql.functions import isnan, when, count, col
        null_count_df = data.select(
            [count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in feature_list]).toPandas()
        null_count_df.index = ['MissingValue']
        describe_df = data.select(feature_list).describe().toPandas()
        describe_df = describe_df.set_index('summary')
        describe_df = describe_df.append(null_count_df) 
        describe_df = describe_df.reset_index()
        return describe_df
    if feature_family == "categorical":
        from pyspark.sql.functions import trim
        data_cat = data.select(feature_list)
        cat_null_count = []
        freq_dist = {}
        for col_ in data_cat.columns:
            print("Processing {}".format(col_))
            data_cat = data_cat.withColumn(col_, trim(data_cat[col_]))
            freq_dist_df = data_cat.groupby(col_).count().toPandas()
            count = data_cat.filter((data_cat[col_] == "")|(data_cat[col_].isNull())).count()
            cat_null_count.append((col_, count))
            freq_dist[col_] = freq_dist_df
        cat_null_count_df = pd.DataFrame(cat_null_count, columns=['Feature', 'MissingValue'])
        total_lines = data_cat.count()
        cat_null_count_df['%MissingValue'] = (cat_null_count_df['MissingValue'] / total_lines) * 100
        cat_null_count_df['%MissingValue'] = cat_null_count_df['%MissingValue'].round(2)
        return freq_dist, cat_null_count_df
    else:
        print("Only two values are possible: \n1. continuos \n2. categorical")

In [4]:
def date_formating(data, dt_col_list):
    """
    Type cast date (StringType) "yyyyMMdd" to DateType
    
    Parameters:
        data (Pyspark DataFrame)
        dt_col_list (list): List of columns that need to be type caste
        
    Returns:
        DataFrame (pySpark)
    """
    try:
        # Formating Date Columns form string to date type
        for dt_col in dt_col_list:
            try:
                data = data.withColumn(dt_col,F.from_unixtime(F.unix_timestamp(dt_col, 'yyyyMMdd')))
                data = data.withColumn(dt_col, F.to_date(F.col(dt_col)))
            except:
                print("Can't format {}".format(dt_col))
        return data
    except Exception as e:
        print(e)

In [5]:
def data_wrangling(data, col_to_trim, date_col):
    """
    This function includes,
        ○ Trimming of all the columns present in the data.
        ○ Creation of srk_clm_id_rj (first 26 digit) from srk_clm_id (29 digit).
        ○ Casting of cl_line_item_chrg_amt & cl_unit_billed from StringType to DoubleType (Float).
        ○ Creation of servicing_npi, rule -> When cl_rend_prov_npi is null or empty populate
           clm_bill_prov_npi otherwise populate cl_rend_prov_npi.
        ○ Creation of cl_emergency_ind, rule -> When cl_emergency_ind =  "Y" populate 1 otherwise 0.

    Parameters:
        data (Pyspark Dataframe).
        col_to_trim (list):  List of columns.
        date_col (list): List of columns that need to be type caste
        
    Returns:
        DataFrame (pySpark)
    """
    for col_ in col_to_trim:
        data = data.withColumn(col_, F.trim(data[col_]))
    data = data.withColumn("srk_clm_id_rj", F.substring(F.col("srk_clm_id"), 1, 26))
    data = data.withColumn("cl_line_item_chrg_amt", F.col("cl_line_item_chrg_amt").cast(DoubleType()))
    data = data.withColumn("cl_units_billed", F.col("cl_units_billed").cast(DoubleType()))
    data = date_formating(data, date_col)
    data = data.withColumn("servicing_npi", 
                                   F.when(((F.col("cl_rend_prov_npi")=="") | F.col("cl_rend_prov_npi").isNull()),
                                          F.col("clm_bill_prov_npi")).otherwise(F.col("cl_rend_prov_npi")))
    data = data.withColumn("cl_emergency_ind", F.when(F.col("cl_emergency_ind") == "Y", 
                                                              F.lit(1)).otherwise(F.lit(0)))
    return data

In [6]:
@udf("int")
def npi_validator(npi):
    """
    Valid NPI Checker, Rule for valid NPI: length of npi == 10, and starting digit should be 1 or 2.
    
    Parameter:
        npi (str).
        
    Returns:
        1 if the NPI is valid else False 0.
    """
    try:
        if (len(npi) == 10) & ((npi[0] == "2") | (npi[0] == "1")):
            return 1
        else:
            return 0
    except:
        return 0

In [7]:
@udf("int")
def place_of_srcv_validator(pls):
    """
    Check for valid place of service (00 to 99)
    
    Parameters:
        pls (string)
    
    Returns:
        1 if the place of service is valid else 0.
    """
    try:
        if (pls in pls_lookup_list):
            return 1
        else:
            return 0
    except:
        return 0

In [3]:
# Configuration
home_path = "/mapr/datalake/uhc/ei/pi_ara/sherlock/development/Provider-Signature-Monitoring/"
## Input
raw_table          = "pi_usrs.psm_837p_m"
eda_report         = 'data/EDA_report_scoring_m.xlsx'
eda_freq_dist      = 'data/EDA_freq_dist_scoring_m.xlsx'
## Temp Table
garbage_clm_table  = "pi_usrs.psm_garbage_clm_dp_m"
good_clm_table     = "pi_usrs.psm_good_clm_dp_m"
## Output table
preprocessed_table = "pi_usrs.psm_preprocessed_data_m"

In [4]:
try:
    os.chdir(home_path)
except Exception as e:
    print(e)

In [10]:
try:
    sc.stop()
except Exception as e:
    print(e)
try:
    sessn.stop()
except Exception as e:
    print(e)
try:
    sc.SparkContext.stop()
except Exception as e:
    print(e)

name 'sessn' is not defined
'SparkContext' object has no attribute 'SparkContext'


In [11]:
conf = pyspark.SparkConf().setAll([('spark.executor.memory', '4g'), 
                                   ('spark.executor.instances', '30'),
                                   ('spark.executor.cores', '4'), 
                                   ('spark.driver.cores','4'), 
                                   ('spark.driver.memory','12g'),
                                   ('spark.yarn.queue', 'araadh_q1.arapi_sq1'),
                                   ('spark.driver.maxResultSize', '2g'),
                                   ('spark.yarn.executor.memoryOverhead', '2g')])
sessn = SparkSession.builder.appName("HPP").config(conf=conf).enableHiveSupport().getOrCreate().newSession()
sessn

### Data Acquisition

In [12]:
raw_data = sessn.table(raw_table)

In [13]:
raw_data_lc = raw_data.count()
print("{} claim lines present in raw data".format(raw_data_lc))
raw_data.select(F.min("batch_run_dt"), F.max("batch_run_dt")).show()

1234284250 claim lines present in raw data
+-----------------+-----------------+
|min(batch_run_dt)|max(batch_run_dt)|
+-----------------+-----------------+
|         20190101|         20200331|
+-----------------+-----------------+



### Data Wrangling

In [14]:
col_to_trim = raw_data.columns

In [15]:
data_to_work = data_wrangling(data = raw_data, col_to_trim = col_to_trim, date_col = ["batch_run_dt"])

In [16]:
data_to_work.select(F.min("batch_run_dt"), F.max("batch_run_dt")).show()

+-----------------+-----------------+
|min(batch_run_dt)|max(batch_run_dt)|
+-----------------+-----------------+
|       2019-01-01|       2020-03-31|
+-----------------+-----------------+



In [17]:
col_to_select = ['prov_tin',
                 'batch_run_dt',
                 'cl_proc_cd',
                 'fln_nbr',
                 'cl_proc_mod1',
                 'cl_proc_mod2',
                 'cl_proc_mod3',
                 'cl_proc_mod4',
                 'cl_line_item_chrg_amt',
                 'cl_place_of_srcv',
                 'cl_emergency_ind',
                 'cl_units_billed',
                 'srk_clm_id_rj',
                 'servicing_npi']
data_to_work = data_to_work.select(col_to_select)

### EDA

In [19]:
writer = pd.ExcelWriter(eda_report)
writer_dist = pd.ExcelWriter(eda_freq_dist)

In [20]:
continuos_var = [c[0] for c in data_to_work.dtypes if c[1]=='double']
categorical_var = [c[0] for c in data_to_work.dtypes if ((c[1]=='string')& ('id' not in c[0]))]
categorical_var, continuos_var

(['prov_tin',
  'cl_proc_cd',
  'fln_nbr',
  'cl_proc_mod1',
  'cl_proc_mod2',
  'cl_proc_mod3',
  'cl_proc_mod4',
  'cl_place_of_srcv',
  'servicing_npi'],
 ['cl_line_item_chrg_amt', 'cl_units_billed'])

In [21]:
eda_continuos = get_eda(data_to_work, continuos_var, "continuos")
eda_continuos

index cl_line_item_chrg_amt     cl_units_billed
0         count            1234280787          1234280089
1          mean     231.1778552368953   4.718336321581869
2        stddev     5256.136885804546  1432.6372574582285
3           min                -612.0                 0.0
4           max           9.2112505E7         2.9587077E7
5  MissingValue                  3463                4161

In [22]:
eda_continuos.to_excel(writer, index= False, sheet_name = "continuos")

In [24]:
categorical_var.remove("fln_nbr")

In [26]:
eda_categorical = get_eda(data_to_work, categorical_var, "categorical")

Processing prov_tin
Processing cl_proc_cd
Processing cl_proc_mod1
Processing cl_proc_mod2
Processing cl_proc_mod3
Processing cl_proc_mod4
Processing cl_place_of_srcv
Processing servicing_npi


In [27]:
eda_categorical_desc = eda_categorical[1]

In [28]:
out = []
for c in list(eda_categorical[0]):
    unique_cnt = eda_categorical[0][c].shape[0]
    out.append((c, unique_cnt))
out_df = pd.DataFrame(out, columns = ['Feature', 'UniqueCount'])
eda_categorical_desc = eda_categorical_desc.merge(out_df, on = "Feature", how = "inner")
eda_categorical_desc

Feature  MissingValue  %MissingValue  UniqueCount
0          prov_tin             0           0.00       618107
1        cl_proc_cd          4217           0.00        28658
2      cl_proc_mod1     838034252          67.90         1069
3      cl_proc_mod2    1160640541          94.03          841
4      cl_proc_mod3    1225935905          99.32          574
5      cl_proc_mod4    1233758680          99.96          366
6  cl_place_of_srcv          4185           0.00          345
7     servicing_npi       7357597           0.60      2033247

In [29]:
for c in list(eda_categorical[0]):
    eda_categorical[0][c].to_excel(writer_dist, index= False, sheet_name = c)
eda_categorical_desc.to_excel(writer, index= False, sheet_name = "categorical")

In [30]:
writer.close()
writer_dist.close()

### Data Cleaning

##### Dropping claims having nan,  null or negative entry for, 
- cl_line_item_chrg_amt
- cl_unit_billed

In [57]:
c = "cl_line_item_chrg_amt"
print("Dropping claims having nan, null or negative entry for {}".format(c))
# Filtering out garbage claims
dump_data = data_to_work.filter((F.isnan(c) | F.col(c).isNull() | (F.col(c) < 0)))
garbage_clms = dump_data.select("srk_clm_id_rj").distinct()
# Adding flag for identification
garbage_clms = garbage_clms.withColumn("Flag", F.lit(c))
# Overwriting the garbage claims to garbage_clm_table
garbage_clms.write.mode("overwrite").format("parquet").saveAsTable(garbage_clm_table)

Dropping claims having nan, null or negative entry for cl_line_item_chrg_amt


In [58]:
c = "cl_units_billed"
print("Dropping claims having nan, null or negative entry for {}".format(c))
# Filtering out garbage claims
dump_data = data_to_work.filter((F.isnan(c) | F.col(c).isNull() | (F.col(c) < 0)))
garbage_clms = dump_data.select("srk_clm_id_rj").distinct()
# Adding flag for identification
garbage_clms = garbage_clms.withColumn("Flag", F.lit(c))
# Appending the garbage claims to garbage_clm_table
garbage_clms.write.mode("append").format("parquet").saveAsTable(garbage_clm_table)

Dropping claims having nan, null or negative entry for cl_units_billed


##### Dropping claims having null or blank or where the length is not equal to 10 or starting digit is not 1 nor 2 for the servicing_npi

In [60]:
c = "servicing_npi"
print("Dropping claims having null, blank or where length is not equal to 10 or starting digit is not 1 nor 2 for {}".format(c))
# Running validation check for NPI
data_to_work = data_to_work.withColumn("valid_npi", npi_validator(c)) 
# Filtering out garbage claims
dump_data = data_to_work.filter(((data_to_work[c] == "") | (data_to_work[c].isNull()) | (data_to_work["valid_npi"] == 0)))
garbage_clms = dump_data.select("srk_clm_id_rj").distinct()
# Adding flag for identification
garbage_clms = garbage_clms.withColumn("Flag", F.lit(c))
# Appending the garbage claims to garbage_clm_table
garbage_clms.write.mode("append").format("parquet").saveAsTable(garbage_clm_table)
data_to_work = data_to_work.drop("valid_npi")

Dropping claims having null, blank or where length is not equal to 10 or starting digit is not 1 nor 2 for servicing_npi


##### Dropping claims having "null" or "blank" & value other than 01 to 99 for cl_place_of_srvc

In [61]:
c = "cl_place_of_srcv"
# Creating the place of service lookup list having elements ranging from 01 to 99.
pls_lookup_list = ["0" + str(i) for i in range(1, 10)] + [str(i) for i in range(1,100)]
print("Dropping claims having null or blank or value other than 01 to 99 {}".format(c))
# Running validation check for place of service.
data_to_work = data_to_work.withColumn("valid_pls", place_of_srcv_validator(c)) 
# Filtering out garbage claims
dump_data = data_to_work.filter(((data_to_work[c] == "") | (data_to_work[c].isNull()) | (data_to_work["valid_pls"] == 0)))
garbage_clms = dump_data.select("srk_clm_id_rj").distinct()
# Adding flag for identification
garbage_clms = garbage_clms.withColumn("Flag", F.lit(c))
# Appending garbage claims to garbage_clm_table
garbage_clms.write.mode("append").format("parquet").saveAsTable(garbage_clm_table)
data_to_work = data_to_work.drop("valid_pls")

Dropping claims having null or blank or value other than 01 to 99 cl_place_of_srcv


##### Dropping claims where proc_mod_cd length is not equal to 2 

In [62]:
# Generating the list of modifier columns
mod_col_list = [col_ for col_ in data_to_work.columns if "proc_mod" in col_]
for c in mod_col_list:
    print("Dropping claims where length of {} is not equal to 2". format(c))
    mod_dump = data_to_work.select(["srk_clm_id_rj", c])
    # Filtering out the garbage claims
    dump_data = mod_dump.filter((mod_dump[c] != "") & (~mod_dump[c].isNull()) & (F.length(F.col(c)) !=2))
    garbage_clms = dump_data.select("srk_clm_id_rj").distinct()
    # Adding flag for identification
    garbage_clms = garbage_clms.withColumn("Flag", F.lit(c))
    # Appending garbage claims to garbage_clm_table
    garbage_clms.write.mode("append").format("parquet").saveAsTable(garbage_clm_table)

Dropping claims where length of cl_proc_mod1 is not equal to 2
Dropping claims where length of cl_proc_mod2 is not equal to 2
Dropping claims where length of cl_proc_mod3 is not equal to 2
Dropping claims where length of cl_proc_mod4 is not equal to 2


##### Dropping claims having null or blank entry for proc_cd

In [63]:
c = "cl_proc_cd"
print("Dropping claims having Null, blank entry for {}".format(c))
# Filtering out garbage claims
dump_data = data_to_work.filter(((data_to_work[c] == "")|(data_to_work[c].isNull())))
garbage_clms = dump_data.select("srk_clm_id_rj").distinct()
# Adding flag for identification
garbage_clms = garbage_clms.withColumn("Flag", F.lit(c))
# Appending garbage claims to garbage_clm_table
garbage_clms.write.mode("append").format("parquet").saveAsTable(garbage_clm_table)

Dropping claims having Null, blank entry for cl_proc_cd


##### Cleaning the data

In [66]:
data_to_work.createOrReplaceTempView("data_to_work")
garbage_claims = get_data(sessn, "select distinct(srk_clm_id_rj) from "+ garbage_clm_table)
all_claims = get_data(sessn, "select distinct(srk_clm_id_rj) from "+ "data_to_work")
good_claims = all_claims.subtract(garbage_claims)
good_claims.write.mode("overwrite").format("parquet").saveAsTable(good_clm_table)
data_to_work = data_to_work.join(good_claims, on="srk_clm_id_rj", how ="inner")

##### Saving the output

In [67]:
data_to_work.repartition(1000, "servicing_npi").write.mode("overwrite").format("parquet").saveAsTable(preprocessed_table)

#### Sanity Check

In [68]:
preprocessed_data = sessn.table(preprocessed_table)

In [69]:
pd.DataFrame(preprocessed_data.dtypes)

0       1
0           srk_clm_id_rj  string
1                prov_tin  string
2            batch_run_dt    date
3              cl_proc_cd  string
4                 fln_nbr  string
5            cl_proc_mod1  string
6            cl_proc_mod2  string
7            cl_proc_mod3  string
8            cl_proc_mod4  string
9   cl_line_item_chrg_amt  double
10       cl_place_of_srcv  string
11       cl_emergency_ind     int
12        cl_units_billed  double
13          servicing_npi  string

In [72]:
preprocessed_data_lc = preprocessed_data.count()
print("{} claim lines present in raw data".format(preprocessed_data_lc))

1226727637 claim lines present in raw data


In [74]:
preprocessed_data_clmcnt = preprocessed_data.select("srk_clm_id_rj").distinct().count()
print("{} claim present in raw data".format(preprocessed_data_clmcnt))

569688048 claim present in raw data


In [78]:
continuos_var = [c[0] for c in preprocessed_data.dtypes if c[1]=='double']
categorical_var = [c[0] for c in preprocessed_data.dtypes if ((c[1]=='string')& ('id' not in c[0]))]
categorical_var.remove("fln_nbr")
categorical_var, continuos_var

(['prov_tin',
  'cl_proc_cd',
  'cl_proc_mod1',
  'cl_proc_mod2',
  'cl_proc_mod3',
  'cl_proc_mod4',
  'cl_place_of_srcv',
  'servicing_npi'],
 ['cl_line_item_chrg_amt', 'cl_units_billed'])

In [80]:
eda_continuos = get_eda(preprocessed_data, continuos_var, "continuos")
print(eda_continuos)
eda_categorical = get_eda(preprocessed_data, categorical_var, "categorical")
print(eda_categorical[1])

          index cl_line_item_chrg_amt     cl_units_billed
0         count            1226727637          1226727637
1          mean    231.38237250704847   4.691261450423328
2        stddev     5269.807220236828  1405.6323874813818
3           min                   0.0                 0.0
4           max           9.2112505E7         2.9587077E7
5  MissingValue                     0                   0
Processing prov_tin
Processing cl_proc_cd
Processing cl_proc_mod1
Processing cl_proc_mod2
Processing cl_proc_mod3
Processing cl_proc_mod4
Processing cl_place_of_srcv
Processing servicing_npi
            Feature  MissingValue  %MissingValue
0          prov_tin             0           0.00
1        cl_proc_cd             0           0.00
2      cl_proc_mod1     832527694          67.87
3      cl_proc_mod2    1153446229          94.03
4      cl_proc_mod3    1218385117          99.32
5      cl_proc_mod4    1226203001          99.96
6  cl_place_of_srcv             0           0.00
7     servi

##### Procedure code

In [ ]:
# query = "select srk_clm_id, cl_fst_serv_dt, cl_lst_serv_dt, cl_rproc_cd from pi_prod_"

##### Closing the Session

In [ ]:
try:
    sc.stop()
except Exception as e:
    print(e)
try:
    sessn.stop()
except Exception as e:
    print(e)
try:
    sc.SparkContext.stop()
except Exception as e:
    print(e)

'SparkContext' object has no attribute 'SparkContext'
